In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import os
import argparse
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
from matplotlib.colors import Normalize
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

from quantnn.qrnn import QRNN

import sys
sys.path.append('../visualize')
from sample_plots import plotRandomSample
sys.path.append('../src')
from load_data import GOESRETRIEVALSDataset, RandomLog, Mask, RandomCrop, Standardize, ToTensor


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device: ', device)

device:  cpu


/home/ingrid/anaconda3/envs/geostat/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729062494/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
channels = list(range(8,17))
channels.remove(12)
fillvalue = -1
quantiles = np.linspace(0.01, 0.99, 99)

In [3]:
net_name = 'boxes_one'#'xception' #'boxes_one_modified' 
BATCH_SIZE = 256 #64 #256
uselog = True #True
n_epochs = 10
lr = 0.001
filename_extension ='_'+str(n_epochs)+'_'+str(lr)
filename = net_name + str(uselog) + str(BATCH_SIZE) 
print(filename)

boxes_oneTrue256


In [4]:
if (net_name == 'boxes_one'):
	from models.boxes_one import Net
	net = Net(len(quantiles), len(channels))
elif (net_name == 'xception'):
	from quantnn.models.pytorch.xception import XceptionFpn
	net =  XceptionFpn(len(channels), quantiles.size, n_features=128)

In [5]:
channels = list(range(8,17))
channels.remove(12)

quantiles = np.linspace(0.01, 0.99, 99)

fillvalue = -1

path_to_val_data = '../dataset/data/dataset-boxes/validation/npy_files'
path_to_stats = '../dataset/data/dataset-boxes/train/stats.npy'
path_to_results = '../results'

In [6]:
qrnn_model = QRNN.load(os.path.join(path_to_results, filename, 'saved_models', filename+ filename_extension+'.pckl'))
Y = np.loadtxt(os.path.join(path_to_results, filename, 'preds', filename+filename_extension+'.txt'))

AttributeError: cannot set attribute '_close' on a 'Dataset' object. Use __setitem__ styleassignment (e.g., `ds['name'] = ...`) instead of assigning variables.

In [ ]:
def plotPerformance(y_true, y_pred, uselog, save=None):

    bins = np.logspace(-2, 2, 100)
    if uselog:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)
    freqs, _, _ = np.histogram2d(y_true, y_pred, bins=bins)

    f, ax = plt.subplots(figsize=(8, 9))

    p = ax.pcolormesh(bins, bins, freqs.T)
    ax.set_xlim([1e-2, 1e2])
    ax.set_ylim([1e-2, 1e2])
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("Reference rain rate [mm / h]")
    ax.set_ylabel("Predicted rain rate [mm / h]")
    ax.plot(bins, bins, c="grey", ls="--")
    f.colorbar(p, ax=ax, orientation="horizontal", label="Surface precipitation [mm / h]")
    ax.set_aspect(1.0)

    plt.tight_layout()
    if not save == None:
        plt.savefig(save)
    plt.show()


In [ ]:
plotPerformance(Y[:,0], Y[:,1], uselog, 
                os.path.join(path_to_results, 'images', filename+ filename_extension+'.png'))

In [ ]:
plt.hist(np.exp(Y[:,0]), bins=500)
plt.show()

In [ ]:
def importData(channels, BATCH_SIZE, path_to_data, path_to_stats, uselog):
    tran = [Mask()]
    if uselog:
        tran.append(RandomLog())
    tran.extend([RandomCrop(128), Standardize(path_to_data, path_to_stats, channels), ToTensor()])
    dataset = GOESRETRIEVALSDataset(
        path_to_data = path_to_data,
        channels = channels, 
        transform = transforms.Compose(tran))
    
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    return(dataset, dataloader)

validation_dataset, validation_data  = importData(channels, BATCH_SIZE, 
                                                  path_to_val_data, path_to_stats, uselog)


In [ ]:
plotRandomSample(validation_dataset,  channels=channels, qrnn=qrnn_model,
                 device=device, cha=[8, 13], uselog=uselog)